In [1]:
import setuptools
import pandas as pd
import numpy as np
import itertools
from pprint import pprint
import timeit
import functools
import operator
from IPython.display import display
import xlsxwriter
import psutil
#Path is variable at top in Change cell open excel
from pyexcelerate import Workbook

In [2]:
#display all of dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
#CHANGE THESE PER FILE

month_day =  '03_09'

nba = "C:\\DATADFS\\" + month_day + "\\DFS_2019_"+ month_day + ".xls"
nba_ids = "C:\\DATADFS\\" + month_day + "\\Export_2019_" + month_day + ".csv"
path = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_Lines.xlsx"
#cvs_path = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_LinesC.csv"
#cvs_path_gpp = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_Lines_GPP.csv"
path_cheatsheet = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_CHTSHT.xlsx"
nba_1 = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_SLTPLYRS.xlsx"

In [68]:
#Main Slate and CSV2 sort names on both list line up and concat
players = pd.read_excel(nba_1)
players = players.reset_index(drop=True)
players = players.replace(np.nan,'')
players = players.sort_values(by=['Name'])
players = players.reset_index(drop=True)
display(players.head())

ids = pd.read_csv(nba_ids)
ids = ids.reset_index(drop=True)
ids = ids.replace(np.nan,'')
#ids.head()

ids['full_names'] = ids['first_name'] + ' ' + ids['last_name']

display(ids.head(300))

dfid = pd.concat([ids[['full_names','fanduel_id']]], axis=1)
dfid.head()
dfid = dfid.sort_values(by=['full_names'])
dfid = dfid.reset_index(drop = True)
display(dfid.head(10000))

#Combine names and id numbers
df = pd.concat([players[['Name']],dfid[['fanduel_id']],players[['Price','Ratio', 'Value','ActualV','ValueC','PickJ','PickG','8x','m/g','toV','Pos','Inj']]], axis=1)
df = df.reset_index(drop=True)
display(df.head(300))

,Rank,Price,Ratio,T6,TSC,Value,ActualV,ValueC,PickJ,PickG,Name,Inj,Team,Pos,Depth,g,m/g,Opp,xG,2x,3x,4x,5x,6x,7x,8x,pV,rV,aV,sV,bV,toV
0,14,7100,50.860563,42.6,-6.489,36.111,0,-1.672552,BOTH,,Al Horford,,BOS,C,,1,31,@ LAL,2,100,50,50,50,50,0,0,14.14,9.936,6.855,2.79,4.08,-1.69
1,41,4900,49.706122,29.4,-5.044,24.356,0,0.585964,,,Al-Farouq Aminu,,POR,PF,,1,29,PHO,2,100,50,50,0,0,0,0,9.06,8.916,2.940,3.12,1.32,-1.00
2,92,3900,21.220513,23.4,-15.124,8.276,0,-1.603479,,,Alex Caruso,,LAL,PG,,1,12,BOS,2,50,50,50,50,50,50,0,3.19,1.716,2.310,1.47,0.57,-0.98
3,17,4700,71.300000,28.2,5.311,33.511,0,-2.738328,BOTH,,Alex Len,,ATL,C,,1,30,BKN,2,100,50,50,50,50,0,0,15.25,11.076,2.715,1.65,5.22,-2.40
4,61,3800,50.765789,22.8,-3.509,19.291,0,1.474771,,,Allen Crabbe,,BKN,SG,,1,26,@ ATL,2,100,0,0,0,0,0,0,10.71,4.536,1.695,2.13,1.29,-1.07


,id,last_name,first_name,team,position,status,opponent,minutes,points,threes,threes_attempted,rebounds,assists,steals,blocks,turnovers,twos,free throws,free_throws_missed,field goals,field_goals_missed,double doubles,triple doubles,usage,odds_spread,odds_total,draftkings_price,draftkings_positions,draftkings_id,fanduel_price,fanduel_positions,fanduel_id,yahoo_price,yahoo_positions,yahoo_id,draftday_price,draftday_positions,draftday_id,fantasydraft_price,fantasydraft_positions,fantasydraft_id,fantasyfeud_price,fantasyfeud_positions,fantasyfeud_id,starfantasyleagues_price,starfantasyleagues_positions,starfantasyleagues_id,fantasyaces_price,fantasyaces_positions,fantasyaces_id,sportstradex_price,sportstradex_positions,sportstradex_id,fantasyscore_price,fantasyscore_positions,fantasyscore_id,fantasyup_price,fantasyup_positions,fantasyup_id,victiv_price,victiv_positions,victiv_id,draftster_price,draftster_positions,draftster_id,fanday_price,fanday_positions,fanday_id,moneyball_price,moneyball_positions,moneyball_id,owntheplay_price,owntheplay_positions,owntheplay_id,fantasyhub_price,fantasyhub_positions,fantasyhub_id,draftpot_price,draftpot_positions,draftpot_id,draftfury_price,draftfury_positions,draftfury_id,draftstars_price,draftstars_positions,draftstars_id,rosters_price,rosters_positions,rosters_id,draft_price,draft_positions,draft_id,draftboard_price,draftboard_positions,draftboard_id,starstreet_price,starstreet_positions,starstreet_id,draftstreet_price,draftstreet_positions,draftstreet_id,fanthrowdown_price,fanthrowdown_positions,fanthrowdown_id,full_names
0,5052,Towns,Karl-Anthony,MIN,C,,WAS,35.0,32.54,2.68,6.05,14.01,4.48,1.16,1.83,3.17,9.19,6.12,1.23,11.87,8.78,0.84,0.025,30.40,MIN by 7.0,241,10400,C,837030,11600,C,58462,50,C,5432,0,,837030,0,C,837030,0,,Karl-Anthony Towns,0,,21205,0,,43684,0,,,11200,C,9681,0,,5852a42d2364510011044388,0,,Karl-Anthony_Towns_Min,0,,ab532a66-9314-4d57-ade7-bb54a70c65ad,0,,,11600,C,Karl-Anthony Towns,0,,,0,,,0,,ab532a66-9314-4d57-ade7-bb54a70c65ad,0,,,22550,PF;C,7001367,0,,5852a42d2364510011044388,0,,,10700,C,Karl-Anthony Towns,0,,,0,,,0,,,Karl-Anthony Towns
1,3701,Antetokounmpo,Giannis,MIL,SF,,CHA,34.0,27.59,1.02,2.90,13.46,6.82,1.31,1.73,3.57,9.39,5.75,2.22,10.41,6.69,0.80,0.088,30.31,MIL by 12.5,227,11300,SF;PF,739957,12100,SF,40199,54,PF,5185,0,,739957,0,PF,739957,0,,Giannis Antetokounmpo,0,,7906,0,,19383,0,,,11800,F,6714,0,,5852a42f2364510011044418,0,,Giannis_Antetokounmpo_Mil,0,,6c60282d-165a-4cba-8e5a-4f2d9d4c5905,0,,,12100,SF,Giannis Antetokounmpo,0,,,0,,,0,,6c60282d-165a-4cba-8e5a-4f2d9d4c5905,0,,,20520,SF;PF,7000949,0,,5852a42f2364510011044418,0,,,11300,SF,Giannis Antetokounmpo,0,,13741,0,,5185,0,,Giannis Antetokounmpo,Giannis Antetokounmpo
2,3439,Beal,Bradley,WAS,SG,,@ MIN,40.0,29.20,2.98,8.28,5.74,7.28,1.63,0.68,3.53,7.46,5.34,1.00,10.44,11.44,0.31,0.022,29.46,MIN by 7.0,241,9700,SG,606912,10600,SG,15595,47,SG,5009,0,,606912,0,SG,606912,0,,Bradley Beal,0,,3856,0,,19188,0,,,9000,G,6950,0,,5852a44123645100110449d1,0,,Bradley_Beal_Was,0,,ff461754-ad20-4eeb-af02-2b46cc980b24,0,,,10600,SG,Bradley Beal,0,,,0,,,0,,ff461754-ad20-4eeb-af02-2b46cc980b24,0,,,19780,SG;SF,7000967,0,,5852a44123645100110449d1,0,,,9900,SG,Bradley Beal,0,,8073,0,,5009,0,,Bradley Beal,Bradley Beal
3,3369,Irving,Kyrie,BOS,PG,Probable,@ LAL,36.0,23.95,2.62,6.29,5.75,7.46,1.92,0.45,2.42,6.47,3.15,0.43,9.09,9.90,0.31,0.022,28.07,,,9000,PG,551768,9500,PG,14498,41,PG,4840,0,,551768,0,PG,551768,0,,Kyrie Irving,0,,3486,0,,19347,0,,,9600,G,6518,0,,5852a4462364510011044b5f,0,,Kyrie_Irving_Cle,0,,dd146010-902b-4ad7-b98c-650d0363a2f0,0,,,9500,PG,Kyrie Irving,0,,,0,,,0,,dd146010-902b-4ad7-b98c-650d0363a2f0,0,,,17270,PG,7001221,0,,5852a4462364510011044b5f,0,,,8900,PG,Kyrie Irving,0,,6268,0,,4840,0,,Kyrie Irving,Kyrie Irving
4,2011,James,LeBron,LAL,SF,,BOS,32.0,23.46,2.04,5.60,8.44,7.70,0.94,0.55,3.33,6.34,4.66,2.21,8.38,8.42,0.38,0.118,29.58,,,10800,SF;PF,214152,11400,SF,9488,50,PF,3704,0,,214152,0,SF,214152,0,,LeBron James,0,,3

,full_names,fanduel_id
0,Al Horford,9453
1,Al-Farouq Aminu,12342
2,Alex Caruso,23981
3,Alex Len,15814
4,Allen Crabbe,15501
5,Andrew Wiggins,49116
6,Anthony Tolliver,9920
7,Aron Baynes,37931
8,Bismack Biyombo,14505
9,Bobby Portis,42156


,Name,fanduel_id,Price,Ratio,Value,ActualV,ValueC,PickJ,PickG,8x,m/g,toV,Pos,Inj
0,Al Horford,9453,7100,50.860563,36.111,0,-1.672552,BOTH,,0,31,-1.69,C,
1,Al-Farouq Aminu,12342,4900,49.706122,24.356,0,0.585964,,,0,29,-1.00,PF,
2,Alex Caruso,23981,3900,21.220513,8.276,0,-1.603479,,,0,12,-0.98,PG,
3,Alex Len,15814,4700,71.300000,33.511,0,-2.738328,BOTH,,0,30,-2.40,C,
4,Allen Crabbe,15501,3800,50.765789,19.291,0,1.474771,,,0,26,-1.07,SG,
5,Andrew Wiggins,49116,6600,48.366667,31.922,0,0.218722,,TS,0,35,-1.95,SG,
6,Anthony Tolliver,9920,3600,35.047222,12.617,0,0.177555,,,0,18,-0.58,PF,
7,Aron Baynes,37931,3500,37.325714,13.064,0,0.350935,,,0,14,-0.89,C,
8,Bismack Biyombo,14505,3500,36.888571,12.911,0,0.131961,,,0,14,-0.62,C,Questionable - left knee sprain
9,Bobby Portis,42156,6300,47.260317,29.774,0,-1.009496,,TS,0,27,-1.61,PF,


In [69]:
#only select marked picks
picks = ['GPP','BOTH','CASH','TS']
regstr = '|'.join(picks)
dfjg = df[df['PickJ'].str.upper().str.contains(regstr)|df['PickG'].str.upper().str.contains(regstr)] #getallpicks
dfjg = dfjg.reset_index(drop=True)

#display(dfjg.head(100))
noplayers = len(dfjg) #number of picked players

#display(noplayers)
noptg =  35 - noplayers #40 payers max

if noptg > 0:
    #display(noptg)
    df_ratio = df.sort_values('Ratio',ascending=False)
    #display(df_ratio.head(5))
    df_ratio = df_ratio[(~df['PickJ'].str.upper().str.contains(regstr))]#~opposite
    df_ratio = df_ratio[(~df['PickG'].str.upper().str.contains(regstr))]
    display(df_ratio.head(7))
    #display(len(df_ratio))
    addplayers = df_ratio.head(noptg)
    c_selections = [dfjg, addplayers]
    full_selection_r = pd.concat(c_selections)
    full_selection_r = full_selection_r.reset_index(drop=True)
    display(full_selection_r.head(300)) #full_selection_r lineups consistent 50/50, Double-Up
    
    #display(noptg)
    df_risk = df.sort_values('8x',ascending=False)
    #display(df_ratio.head(5))
    df_risk = df_risk[(~df['PickJ'].str.upper().str.contains(regstr))]#~opposite
    df_risk = df_risk[(~df['PickG'].str.upper().str.contains(regstr))]
    display(df_risk.head(7))
    #display(len(df_ratio))
    addplayers = df_risk.head(noptg)
    c_selections = [dfjg, addplayers]
    full_selection_e = pd.concat(c_selections)
    full_selection_e = full_selection_e.reset_index(drop=True)
    display(full_selection_e.head(300)) #full_selection_e riskest lineups explosive GPP grabs 8x value
    
else:
    full_selection_r = dfjg
    full_selection_e = dfjg
    
wb = Workbook()

fsrv = [full_selection_r.columns]+list(full_selection_r.values)
fsev = [full_selection_e.columns]+list(full_selection_e.values)
wb.new_sheet('FS_R', data=fsrv)
wb.new_sheet('FS_E', data=fsev)

wb.save(path_cheatsheet)
    
    
full_selection_r.drop('8x', axis=1, inplace=True)
full_selection_e.drop('8x', axis=1, inplace=True)


    
%xdel picks
%xdel regstr
%xdel dfjg
%xdel noplayers
%xdel noptg
%xdel df_ratio
%xdel addplayers
%xdel c_selections

c:\python27\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\python27\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Name,fanduel_id,Price,Ratio,Value,ActualV,ValueC,PickJ,PickG,8x,m/g,toV,Pos,Inj
27,Ed Davis,12365,3800,55.371053,21.041,0,0.572624,,,0,20,-1.17,PF,
28,Enes Kanter,14516,4400,51.979545,22.871,0,0.437109,,,0,19,-1.18,C,
18,Dario Saric,55056,4300,51.806977,22.277,0,1.089100,,,0,24,-1.03,PF,
4,Allen Crabbe,15501,3800,50.765789,19.291,0,1.474771,,,0,26,-1.07,SG,
67,Marvin Williams,9451,5200,49.792308,25.892,0,-1.681167,,,50,33,-0.69,PF,
50,Josh Okogie,84689,3700,49.745946,18.406,0,1.099735,,,0,26,-1.22,SG,
1,Al-Farouq Aminu,12342,4900,49.706122,24.356,0,0.585964,,,0,29,-1.00,PF,


,Name,fanduel_id,Price,Ratio,Value,ActualV,ValueC,PickJ,PickG,8x,m/g,toV,Pos,Inj
0,Al Horford,9453,7100,50.860563,36.111,0,-1.672552,BOTH,,0,31,-1.69,C,
1,Alex Len,15814,4700,71.300000,33.511,0,-2.738328,BOTH,,0,30,-2.40,C,
2,Andrew Wiggins,49116,6600,48.366667,31.922,0,0.218722,,TS,0,35,-1.95,SG,
3,Bobby Portis,42156,6300,47.260317,29.774,0,-1.009496,,TS,0,27,-1.61,PF,
4,Bradley Beal,15595,10600,47.554717,50.408,0,0.650322,BOTH,,0,40,-3.53,SG,
5,Brook Lopez,9621,6200,50.774194,31.480,0,-1.930333,BOTH,,0,30,-0.85,C,
6,Caris LeVert,23798,6300,46.687302,29.413,0,0.122661,,TS,0,27,-2.34,SG,
7,Cody Zeller,15691,5700,45.666667,26.030,0,1.274949,,TS,0,27,-1.44,C,
8,D'Angelo Russell,58312,9100,48.420879,44.063,0,-0.245129,,TS,0,32,-2.98,SG,
9,Deandre Ayton,84673,8100,45.433333,36.801,0,-2.493666,,TS,0,33,-1.55,C,


c:\python27\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\python27\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Name,fanduel_id,Price,Ratio,Value,ActualV,ValueC,PickJ,PickG,8x,m/g,toV,Pos,Inj
77,Rajon Rondo,9455,7400,43.631081,32.287,0,-2.078358,,,50,34,-3.67,PG,
80,Rodions Kurucs,84700,4900,40.571429,19.880,0,0.543142,,,50,26,-1.56,PF,
67,Marvin Williams,9451,5200,49.792308,25.892,0,-1.681167,,,50,33,-0.69,PF,
34,Gordon Hayward,12366,5100,42.556863,21.704,0,-2.076957,,,50,25,-1.38,SF,
71,Moe Harkless,16178,6000,38.546667,23.128,0,-1.320493,,,0,29,-0.90,SF,
70,Miles Bridges,84678,3600,36.494444,13.138,0,0.232995,,,0,17,-0.56,SF,
69,Mike Muscala,17894,3500,10.677143,3.737,0,0.875305,,,0,5,-0.18,C,


,Name,fanduel_id,Price,Ratio,Value,ActualV,ValueC,PickJ,PickG,8x,m/g,toV,Pos,Inj
0,Al Horford,9453,7100,50.860563,36.111,0,-1.672552,BOTH,,0,31,-1.69,C,
1,Alex Len,15814,4700,71.300000,33.511,0,-2.738328,BOTH,,0,30,-2.40,C,
2,Andrew Wiggins,49116,6600,48.366667,31.922,0,0.218722,,TS,0,35,-1.95,SG,
3,Bobby Portis,42156,6300,47.260317,29.774,0,-1.009496,,TS,0,27,-1.61,PF,
4,Bradley Beal,15595,10600,47.554717,50.408,0,0.650322,BOTH,,0,40,-3.53,SG,
5,Brook Lopez,9621,6200,50.774194,31.480,0,-1.930333,BOTH,,0,30,-0.85,C,
6,Caris LeVert,23798,6300,46.687302,29.413,0,0.122661,,TS,0,27,-2.34,SG,
7,Cody Zeller,15691,5700,45.666667,26.030,0,1.274949,,TS,0,27,-1.44,C,
8,D'Angelo Russell,58312,9100,48.420879,44.063,0,-0.245129,,TS,0,32,-2.98,SG,
9,Deandre Ayton,84673,8100,45.433333,36.801,0,-2.493666,,TS,0,33,-1.55,C,


In [70]:
full_selection_e[['fanduel_id']] = full_selection_e[['fanduel_id']].apply(pd.to_numeric, downcast = 'integer')
full_selection_r[['fanduel_id']] = full_selection_r[['fanduel_id']].apply(pd.to_numeric, downcast = 'integer')

In [71]:
%load_ext cython 
#cython run code in C slightly faster

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [72]:
%%cython
import cython #incoporate cython module cimports
cimport cython
import setuptools
import pandas as pd
import numpy as np
cimport numpy as np
import itertools
import functools
import operator
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)

def comb_function_opti_ulti(df,x=0,y=0,p1=0,p2=0,p3=0,p4=0):
    
    
    #------------Select Positions--------------------------- include in def
    pg = list(itertools.combinations(df.loc[df['Pos'] == 'PG'].values , 2))
    sg = list(itertools.combinations(df.loc[df['Pos'] == 'SG'].values , 2))
    sf = list(itertools.combinations(df.loc[df['Pos'] == 'SF'].values , 2))
    pf = list(itertools.combinations(df.loc[df['Pos'] == 'PF'].values , 2))
    c = list(itertools.combinations(df.loc[df['Pos'] == 'C'].values , 1))

    combine_list = [pg, sg, sf, pf, c]

    true_list = []
    
    #---------------------------------------------------------------
    if y==0: #change the variable to x
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (((sum_combo[2]) <= 60000) and  ((sum_combo[4]) > x)): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval


            else:
                #del combo[:]
                del combo
                
    #---------------------------------------------------------------------        
    elif p1==0: #all players with range
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (((sum_combo[2]) <= 60000) and  ((x <= (sum_combo[4]) <= y))): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval


            else:
                #del combo[:]
                del combo
                
    #-----------------------------------------------------------------------
    elif p2==0: #just one player #50/50
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (sum_combo[2] <= 60000) and  (x <= sum_combo[4] <= y) and (np.any(npcombo[:,1] == p1)): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval

            else:
                #del combo[:]
                del combo
                
    elif p3==0:
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (sum_combo[2] <= 60000) and  (x <= sum_combo[4] <= y) and (np.any(npcombo[:,1] == p1)) and (np.any(npcombo[:,1] == p2)): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval

            else:
                #del combo[:]
                del combo
    
    elif p4==0:
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (sum_combo[2] <= 60000) and  (x <= sum_combo[4] <= y) and (np.any(npcombo[:,1] == p1)) and (np.any(npcombo[:,1] == p2)) and (np.any(npcombo[:,1] == p3)): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval

            else:
                #del combo[:]
                del combo
                                                                                                          
    else: #selected players #50/50' #selected players #50/50'
        for element in itertools.product(*combine_list):
            combo = list(element)
            npcombo = np.concatenate(combo) #flatten

            sum_combo=npcombo.sum(axis=0)

            if (sum_combo[2] <= 60000) and  (x <= sum_combo[4] <= y) and (np.any(npcombo[:,1] == p1)) and (np.any(npcombo[:,1] == p2)) and (np.any(npcombo[:,1] == p3)) and (np.any(npcombo[:,1] == p4)): #(243 <= (sum_combo[4]) <= 274)
                true_list.append(npcombo)
                #del combo[:]
                #del combo
                #2 is salary ------------ 4 is combined projval

            else:
                #del combo[:]
                del combo
            
    #if not true_list:
            
    return true_list

In [73]:
def df_operations_tres(arr):
    if pd.DataFrame(arr).empty:
        df = pd.DataFrame() #check validity of this function
        
    else:
        df = pd.DataFrame(np.array(arr).reshape(-1,13))
        
        #----------Column Titles - Change Number columns,Strings to Int Floats-------------

        df.columns=['Name','FD_id','Price','Ratio','Value','ActualV','ValueC','PickJ','PickG','m/g','toV','Pos','Inj']
        
        df[['FD_id']] = df[['FD_id']].apply(pd.to_numeric, downcast = 'integer')
        df['FD_id'] = df['FD_id'].astype(object)
        df[['Name', 'FD_id', 'Pos', 'Inj','PickJ','PickG']] = df[['Name', 'FD_id', 'Pos', 'Inj','PickJ','PickG']].apply(lambda x: x.astype('category'))
        df[['Ratio', 'Value','ActualV','ValueC','m/g','toV']] = df[['Ratio','Value','ActualV','ValueC','m/g','toV']].apply(pd.to_numeric, downcast = 'float')
        df[['Price', 'm/g']] = df[['Price', 'm/g']].apply(pd.to_numeric, downcast = 'integer')
        
        df['key'] = df.groupby(df.index.values//9).Value.transform(sum)-df.groupby(df.index.values//9).Value.transform(min) #projval
        
        df['Real Val'] = df.groupby(df.index.values//9).ActualV.transform(sum)-df.groupby(df.index.values//9).ActualV.transform(min) #realval
        df['idx'] = df.groupby(df.index.values//9).ngroup() #groupnumber
        df['key2'] = df.Pos.map({'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5}) #locks positions in properly
        df[['idx', 'key2']] = df[['idx', 'key2']].apply(pd.to_numeric, downcast = 'integer')
        #idx and key2 possible downcast

        df = df.sort_values(['key', 'idx', 'key2'], ascending=[False, True, True]) #sorts positions properly locks in properly
        
    
    return df

In [74]:
def fan_duel_lineups(df):
    if not df.empty:
        fdid_list = df['FD_id'].values.tolist()
        df_fanduel_lineup = pd.DataFrame(np.array(fdid_list).reshape(-1,9))
        df_fanduel_lineup.columns=['PG','PG','SG','SG','SF','SF','PF','PF','C']
        #df_fanduel_lineup[['PG','PG','SG','SG','SF','SF','PF','PF','C']] = df_fanduel_lineup[['PG','PG','SG','SG','SF','SF','PF','PF','C']].apply(pd.to_numeric)
        grp = df.groupby(np.arange(len(df.index.values))//9)
        lusum = (grp.sum()-grp.min()).sort_values('Value', ascending = False)
        df_fanduel_lineup = pd.concat([df_fanduel_lineup, lusum[['Value','ActualV']]], axis=1)
        df_fanduel_lineup.rename(columns={df_fanduel_lineup.columns[9]:'Proj Value'}, inplace=True)
        df_fanduel_lineup.rename(columns={df_fanduel_lineup.columns[10]:'Real Value'}, inplace=True)
        
    else:
        df_fanduel_lineup = pd.DataFrame()
    return df_fanduel_lineup

In [75]:
def spread_em_random(df,num=0):
    dffs = df_operations_tres(df)
    dffs_fdl_r = fan_duel_lineups(dffs)
    
    Cdffs =  dffs.head(linecount)
    #display(dfratio.head(500000))
    fsvalues = [Cdffs.columns]+list(Cdffs.values)

    Cdffs_fdl_r = dffs_fdl_r.sample(n=num)
    fdlffsvalues = [Cdffs_fdl_r.columns]+list(Cdffs_fdl_r.values)
    
    return fsvalues, fdlffsvalues

In [76]:
def spread_em(df):
    dffs = df_operations_tres(df)
    dffs_fdl_r = fan_duel_lineups(dffs)
    
    Cdffs =  dffs.head(linecount)
    #display(dfratio.head(500000))
    fsvalues = [Cdffs.columns]+list(Cdffs.values)


    Cdffs_fdl_r = dffs_fdl_r.head(fdlcount)
    fdlffsvalues = [Cdffs_fdl_r.columns]+list(Cdffs_fdl_r.values)
    
    return fsvalues, fdlffsvalues

In [77]:
def themix(p1x,p2x,p3x,p4x=0):
    df7_lc = df7_fd = df3a_lc = df3a_fd = df3b_lc = df3b_fd = df3c_lc = df3c_fd = pd.DataFrame()
    
    set10 = 15+1
    set6 = 25+1
    
    df1 = comb_function_opti_ulti(full_selection_e,230,500,p1x,p2x)
    df2 = comb_function_opti_ulti(full_selection_e,230,500,p1x,p3x)
    df3 = comb_function_opti_ulti(full_selection_e,230,500,p2x,p3x)
    df4 = comb_function_opti_ulti(full_selection_e,230,500,p1x)
    df5 = comb_function_opti_ulti(full_selection_e,230,500,p2x)
    df6 = comb_function_opti_ulti(full_selection_e,230,500,p3x)
    
    
    df1_lc, df1_fd = spread_em(df1)
    df2_lc, df2_fd = spread_em(df2)
    df3_lc, df3_fd = spread_em(df3)
    df4_lc, df4_fd = spread_em(df4)
    df5_lc, df5_fd = spread_em(df5)
    df6_lc, df6_fd = spread_em(df6)
    
    if p4x != 0: #change this
        df7 = comb_function_opti_ulti(full_selection_e,230,280,p4x)
        df3_a = comb_function_opti_ulti(full_selection_e,230,280,p1x,p4x)
        df3_b = comb_function_opti_ulti(full_selection_e,230,280,p2x,p4x)
        df3_c = comb_function_opti_ulti(full_selection_e,230,280,p4x,p3x)
        df7_lc, df7_fd = spread_em(df7)
        df3a_lc, df3a_fd = spread_em(df3_a)
        df3b_lc, df3b_fd = spread_em(df3_b)
        df3c_lc, df3c_fd = spread_em(df3_c)
    
    #return df1_lc, df1_fd, df2_lc, df2_fd, df3_lc, df3_fd, df4_lc, df4_fd, df5_lc, df5_fd, df6_lc, df6_fd, df7_lc, df7_fd, df3a_lc, df3a_fd, df3b_lc, df3b_fd, df3c_lc, df3c_fd 
    print('Line computations.....Done')

    wb.new_sheet('Cash Quin - GPP - 1', data=df1_lc)
    wb.new_sheet('Cash Quin - GPP - L - 1', data=df1_fd)

    wb.new_sheet('Cash Quin -  GPP - 2', data=df2_lc)
    wb.new_sheet('Cash Quin - GPP - L - 2', data=df2_fd)

    wb.new_sheet('Cash Quin -  GPP - 3', data=df3_lc)
    wb.new_sheet('Cash Quin - GPP - L - 3', data=df3_fd)

    wb.new_sheet('Cash Quin -  GPP - 4', data=df4_lc)
    wb.new_sheet('Cash Quin - GPP - L - 4', data=df4_fd)

    wb.new_sheet('Cash Quin -  GPP - 5', data=df5_lc)
    wb.new_sheet('Cash Quin - GPP - L - 5', data=df5_fd)

    wb.new_sheet('Cash Quin -  GPP - 6', data=df6_lc)
    wb.new_sheet('Cash Quin - GPP - L - 6', data=df6_fd)

    if p4x != 0:

        wb.new_sheet('Cash Quin -  GPP - 7', data=df7_lc)
        wb.new_sheet('Cash Quin - GPP - L - 7', data=df7_fd)

        wb.new_sheet('Cash Quin -  GPP - 8', data=df3a_lc)
        wb.new_sheet('Cash Quin - GPP - L - 8', data=df3a_fd)

        wb.new_sheet('Cash Quin -  GPP - 9', data=df3b_lc)
        wb.new_sheet('Cash Quin - GPP - L - 9', data=df3b_fd)

        wb.new_sheet('Cash Quin -  GPP - 10', data=df3c_lc)
        wb.new_sheet('Cash Quin - GPP - L - 10', data=df3c_fd)

    
    print('Sheets.........Done')
    



    if p4x != 0:
        gpp_1 = df1_fd [1:set10]
        gpp_2 = df2_fd[1:set10]
        gpp_3 = df3_fd[1:set10]
        gpp_4 = df4_fd[1:set10]
        gpp_5 = df5_fd[1:set10]
        gpp_6 = df6_fd[1:set10]
        gpp_7 = df7_fd[1:set10]
        gpp_8 = df3a_fd[1:set10]
        gpp_9 = df3b_fd[1:set10]
        gpp_10 = df3c_fd[1:set10]

        mainfdl = gpp_1 + gpp_2 + gpp_3 + gpp_4 + gpp_5 + gpp_6 + gpp_7 + gpp_8 + gpp_9 + gpp_10

    else:
        gpp_1 = df1_fd[1:set6]
        gpp_2 = df2_fd[1:set6]
        gpp_3 = df3_fd[1:set6]
        gpp_4 = df4_fd[1:set6]
        gpp_5 = df5_fd[1:set6]
        gpp_6 = df6_fd[1:set6]

        mainfdl = gpp_1 + gpp_2 + gpp_3 + gpp_4 + gpp_5 + gpp_6

    print('Finishing Final Sheet')
    
    wb.new_sheet('Final - Line ', data = mainfdl)


    wb.save(path)
    
    return

In [78]:
def studmid(p1x,p2x,p3x,p4x,p5x,p6x,num):
    
    set6 = num + 1
    
    df1 = comb_function_opti_ulti(full_selection_e,230,500,p1x,p6x)
    df2 = comb_function_opti_ulti(full_selection_e,230,500,p2x,p6x)
    df3 = comb_function_opti_ulti(full_selection_e,230,500,p3x,p6x)
    df4 = comb_function_opti_ulti(full_selection_e,230,500,p4x,p6x)
    df5 = comb_function_opti_ulti(full_selection_e,230,500,p5x,p6x)
    
    
    df1_lc, df1_fd = spread_em(df1)
    df2_lc, df2_fd = spread_em(df2)
    df3_lc, df3_fd = spread_em(df3)
    df4_lc, df4_fd = spread_em(df4)
    df5_lc, df5_fd = spread_em(df5)
    
    #return df1_lc, df1_fd, df2_lc, df2_fd, df3_lc, df3_fd, df4_lc, df4_fd, df5_lc, df5_fd, df6_lc, df6_fd, df7_lc, df7_fd, df3a_lc, df3a_fd, df3b_lc, df3b_fd, df3c_lc, df3c_fd 
    print('Line computations.....Done')

    wb.new_sheet('Cash Quin - GPP - 1', data=df1_lc)
    wb.new_sheet('Cash Quin - GPP - L - 1', data=df1_fd)

    wb.new_sheet('Cash Quin -  GPP - 2', data=df2_lc)
    wb.new_sheet('Cash Quin - GPP - L - 2', data=df2_fd)

    wb.new_sheet('Cash Quin -  GPP - 3', data=df3_lc)
    wb.new_sheet('Cash Quin - GPP - L - 3', data=df3_fd)

    wb.new_sheet('Cash Quin -  GPP - 4', data=df4_lc)
    wb.new_sheet('Cash Quin - GPP - L - 4', data=df4_fd)

    wb.new_sheet('Cash Quin -  GPP - 5', data=df5_lc)
    wb.new_sheet('Cash Quin - GPP - L - 5', data=df5_fd)
   
    print('Sheets.........Done')
    
    gpp_1 = df1_fd[1:set6]
    gpp_2 = df2_fd[1:set6]
    gpp_3 = df3_fd[1:set6]
    gpp_4 = df4_fd[1:set6]
    gpp_5 = df5_fd[1:set6]

    mainfdl = gpp_1 + gpp_2 + gpp_3 + gpp_4 + gpp_5

    print('Finishing Final Sheet')
    
    wb.new_sheet('Final - Line ', data = mainfdl)

    wb.save(path)
    
    return

In [79]:
%xdel nba
%xdel players
%xdel nba_ids 
%xdel ids
%xdel dfid
%xdel df

In [80]:
%reset -f out
#%reset_selective -f d

Flushing output cache (1 entries)


In [81]:
import gc
gc.collect()

0

In [92]:
#Path is variable at top in Change cell open excel
from pyexcelerate import Workbook
wb = Workbook()

In [83]:
linecount = (150*9) #150 max lineups
fdlcount = 1000

In [ ]:
#run from here up

In [ ]:
%reset -f out
%xdel full_selection_r
%xdel df_ans_fs
%xdel dffs
%xdel Cdffs
%xdel fsvalues
#%xdel dffs_fdl_r
%xdel Cdffs_fdl_r
%xdel fdlffsvalues

# START HERE
# Cash Games - Trip - Quin
- take 40 with selection e(risky line up) 
- Pick stud 10,000 range high value…
- Pick 9,000 IS next bracket range and high RATIO

## PROVEN RESULT
### Quin or GPP
- 2 Studs
- 40/35 or  end of risky list

In [ ]:
#comb_function_opti_ulti(df,x=0,y=0,p1=0,p2=0)
p1x = 9488 #BRON - 9488
p2x = 80810 #Fox - 80810
df_ans_fs = comb_function_opti_ulti(full_selection_e,230,400,p1x,p2x) #230-280

dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)

#----------------------------------
Cdffs =  dffs.head(linecount)
#display(dfratio.head(500000))
fsvalues = [Cdffs.columns]+list(Cdffs.values)


Cdffs_fdl_r = dffs_fdl_r.head(fdlcount)
#dffs_fdl_r.head(100000)
fdlffsvalues = [Cdffs_fdl_r.columns]+list(Cdffs_fdl_r.values)

wb.new_sheet('Cash Quin -  GPP', data=fsvalues)

wb.new_sheet('Cash Quin - GPP - L', data=fdlffsvalues)

#click save

# Three to Four Stud Mix
-  pick 3 to 4 players mix

In [93]:
p1x = 58462 #TOWNS
p2x = 40199 #GIANIIS
p3x = 14498 #KYRIE
themix(p1x,p2x,p3x,p4x=0)

Line computations.....Done
Sheets.........Done
Finishing Final Sheet


# Tailor Made Mid Players with Studs
    -5 Studs
    -Grab Low 1st Teir or top second Teir round 8000-9000( good consistency)
    -TS 2 to 3 top ratio guys

In [46]:
#def studmid(p1x,p2x,p3x,p4x,p5x,p6x,num): p1x-p5x Studs, p6x - mid
p1x = 58462  #Towns
p2x = 9644 #Harden
p3x = 66115 #Simmons
p4x = 14514 #Vuce
p5x = 14498 #kyrie
#p6x = 9524 #curry
p6x = 9623 #paul
num = 18

studmid(p1x,p2x,p3x,p4x,p5x,p6x,num) #p1xp6x p2xp6x ......

Line computations.....Done
Sheets.........Done
Finishing Final Sheet


# GPP Mix-up(e)

- Two studs and take the mix with the fullselect_e from the risk lineups 8x
- Make Stud Judgement off consistency value maybe the been cooking may be not. the number the better.
- Let the rest of the card fall stick to range. Use full "50" with selection e
- only take limit of 8x no zeroes

In [ ]:
#comb_function_opti_ulti(df,x=0,y=0,p1=0,p2=0)
p1x = 9642 #westbook
p2x = 9644 #Towns
df_ans_fs = comb_function_opti_ulti(full_selection_e,230,280,p1x,p2x) #230-280 15 minutes

dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)

df_gpp = dffs_fdl_r.sample(n=150) #random
#----------------------------------

fdlgppcombvalues = [df_gpp.columns]+list(df_gpp.values)

wb.new_sheet('GPP_1', data=fdlgppcombvalues)

In [ ]:
fdl_single_comb = df_gpp
#fdl_single_comb.sort_values('Proj Value', ascending=False, inplace =True)
fdl_single_comb.drop_duplicates(inplace = True)
fdl_single_comb.reset_index(drop=True,inplace=True)


#display(fdl_single_comb.head(40000))
Cfdl_single_comb =  fdl_single_comb.head(150) #how many to take to cvs
Cfdl_single_comb = Cfdl_single_comb.astype(str)
Cfdl_single_comb.dtypes
fdlkey = '33134-'
Cfdl_single_comb['PG'] = fdlkey + Cfdl_single_comb['PG']
Cfdl_single_comb['SG'] = fdlkey + Cfdl_single_comb['SG']
Cfdl_single_comb['SF'] = fdlkey + Cfdl_single_comb['SF']
Cfdl_single_comb['PF'] = fdlkey + Cfdl_single_comb['PF']
Cfdl_single_comb['C'] = fdlkey + Cfdl_single_comb['C']

display(Cfdl_single_comb.head(150))

Cfdl_single_comb.to_csv(cvs_path, index = False)

In [ ]:
#comb_function_opti_ulti(df,x=0,y=0,p1=0,p2=0)
p1x = 9642

df_ans_fs = comb_function_opti_ulti(full_selection_e,270,280,p1x) #280-270 15 minutes
dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)
df_gpp1 = dffs_fdl_r.sample(n=30) #random

df_ans_fs = comb_function_opti_ulti(full_selection_e,260,269,p1x) #269-260
dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)
df_gpp2 = dffs_fdl_r.sample(n=30) #random

df_ans_fs = comb_function_opti_ulti(full_selection_e,250,259,p1x) #259-250
dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)
df_gpp3 = dffs_fdl_r.sample(n=30) #random

df_ans_fs = comb_function_opti_ulti(full_selection_e,240,249,p1x) #249-240
dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)
df_gpp4 = dffs_fdl_r.sample(n=30) #random

df_ans_fs = comb_function_opti_ulti(full_selection_e,230,239,p1x) #239-230
dffs = df_operations_tres(df_ans_fs)
dffs_fdl_r = fan_duel_lineups(dffs)
df_gpp5 = dffs_fdl_r.sample(n=30) #random

#concat
fdl_gpp_lines = [df_gpp1, df_gpp2, df_gpp3, df_gpp4, df_gpp5]
fdl_single_gpp = pd.concat(fdl_gpp_lines)
fdl_single_gpp.sort_values('Proj Value', ascending=False, inplace =True)
#display(fdl_single_comb.head(40000))

In [ ]:
fdlgppcombvalues = [fdl_single_gpp.columns]+list(fdl_single_gpp.values)

wb.new_sheet('GPP_', data=fdlgppcombvalues)

# Save Workbook

In [ ]:
#TEMP
wb.save(path)

# CVS Convert

In [96]:
cvs_path = "C:\\DATADFS\\" + month_day + "\\" + month_day + "output.csv"
readthis = "C:\\DATADFS\\" + month_day + "\\" + month_day + "_Lines.xlsx"
#scan fdlines
names = ['PG', 'PG', 'SG', 'SG', 'SF', 'SF', 'PF', 'PF', 'C','PROJ VAL', 'REAL VAL']
#df_csv = pd.read_excel(readthis, sheet_name='Cash Quin - GPP - L', header=None, names=names) #comment this out
df_csv = pd.read_excel(readthis, sheet_name='Final - Line ', header=None, names=names) #"C:\\DATADFS\\1-12\\1-12_Lines.xlsx"
#df_csv.drop(df_csv.index[0], inplace = True)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df_csv.drop(df_csv.index[0], inplace = True) #comments this out

#df_csv.drop_duplicates(inplace = True)
#df_csv.reset_index(drop=True,inplace=True)
dffs_fdl_r = df_csv
display(df_csv.head(150))
fdl_single_comb = dffs_fdl_r
#fdl_single_comb.sort_values('Proj Value', ascending=False, inplace =True)
fdl_single_comb.drop_duplicates(inplace = True)
fdl_single_comb.reset_index(drop=True,inplace=True)



#display(fdl_single_comb.head(40000))
Cfdl_single_comb =  fdl_single_comb.head(150) #how many to take to cvs - LINEUP
Cfdl_single_comb = Cfdl_single_comb.astype(str)
Cfdl_single_comb.dtypes
fdlkey = '33219-'
Cfdl_single_comb['PG'] = fdlkey + Cfdl_single_comb['PG']
Cfdl_single_comb['SG'] = fdlkey + Cfdl_single_comb['SG']
Cfdl_single_comb['SF'] = fdlkey + Cfdl_single_comb['SF']
Cfdl_single_comb['PF'] = fdlkey + Cfdl_single_comb['PF']
Cfdl_single_comb['C'] = fdlkey + Cfdl_single_comb['C']

display(Cfdl_single_comb.head(150))

Cfdl_single_comb.to_csv(cvs_path, index = False)

,PG,PG,SG,SG,SF,SF,PF,PF,C,PROJ VAL,REAL VAL
0,12355,84671,15559,84704,40199,9654,66236,9451,15814,292.690002,0
1,23097,84671,23798,84704,40199,9654,80808,66236,15814,291.955017,0
2,12355,84671,49116,84704,40199,9654,66236,9451,15814,291.842010,0
3,14498,84671,15595,84704,16363,9654,80808,66236,15814,291.533020,0
4,14498,23097,84704,16359,40199,9654,66236,9451,15814,291.247009,0
5,14498,23097,84704,16359,40199,9654,66236,9451,15814,291.247009,0
6,14498,23097,84704,16359,40199,9654,66236,9451,15814,291.247009,0
7,23097,84671,15559,84704,40199,16363,66236,9451,15814,290.966980,0
8,23097,84671,84704,16359,40199,9654,42156,66236,15814,290.881012,0
9,9446,84671,15559,84704,40199,16363,66236,9451,15814,290.877991,0


,PG,PG,SG,SG,SF,SF,PF,PF,C,PROJ VAL,REAL VAL
0,33219-12355,33219-84671,33219-15559,33219-84704,33219-40199,33219-9654,33219-66236,33219-9451,33219-15814,292.690002441406,0
1,33219-23097,33219-84671,33219-23798,33219-84704,33219-40199,33219-9654,33219-80808,33219-66236,33219-15814,291.955017089844,0
2,33219-12355,33219-84671,33219-49116,33219-84704,33219-40199,33219-9654,33219-66236,33219-9451,33219-15814,291.842010498047,0
3,33219-14498,33219-84671,33219-15595,33219-84704,33219-16363,33219-9654,33219-80808,33219-66236,33219-15814,291.533020019531,0
4,33219-14498,33219-23097,33219-84704,33219-16359,33219-40199,33219-9654,33219-66236,33219-9451,33219-15814,291.247009277344,0
5,33219-23097,33219-84671,33219-15559,33219-84704,33219-40199,33219-16363,33219-66236,33219-9451,33219-15814,290.966979980469,0
6,33219-23097,33219-84671,33219-84704,33219-16359,33219-40199,33219-9654,33219-42156,33219-66236,33219-15814,290.881011962891,0
7,33219-9446,33219-84671,33219-15559,33219-84704,33219-40199,33219-16363,33219-66236,33219-9451,33219-15814,290.877990722656,0
8,33219-14498,33219-84671,33219-58312,33219-84704,33219-9809,33219-9654,33219-80808,33219-66236,33219-15814,290.588012695312,0
9,33219-12355,33219-84671,33219-84704,33219-16359,33219-40199,33219-9654,33219-66236,33219-9451,33219-15814,290.318023681641,0
